We need an AWS account role with SageMaker access. This role is used to give SageMaker access to S3, launch an EC2 instance and send command with Systems Manager.

In [29]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import botocore
import json

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

In [30]:
%store -r
print("instance_id: ", instance_id)

instance_id:  i-0aabb20c139dde1af


In [4]:
# S3 bucket and folders for saving model artifacts.
# Feel free to specify different bucket/folders here if you wish.
folder = 'DEMO-Sandbox-Sagemaker-Edge'
compilation_output_sub_folder = folder + '/compilation-output'
iot_folder = folder + '/iot'

# S3 Location to save the model artifact after compilation
s3_compilation_output_location = 's3://{}/{}'.format(bucket, compilation_output_sub_folder)

Finally we upload the test image to S3 bucket. This image will be used in inference later.

## Inference on Edge

In our example, we will use [AWS Systems Manager](https://docs.aws.amazon.com/systems-manager/latest/userguide/ssm-agent.html) to remotely perform actions on the EC2 instance. To see the SSM logs from CloudWatch, refer to [Install CouldWatch Agent](#(Optional)Install-CloudWatch-Agent). 

Executive status of send command is available in [AWS Systems Manager console](https://console.aws.amazon.com/systems-manager/run-command/complete-commands) command history.

In [5]:
ssm_client = boto3.client('ssm', region_name=region)

### Setup Sagemaker Edge Manager Agent

Download sagemaker edge manager agent binary exaples to EC2 instance.

Please fetch the latest version of binaries from sagemaker edge release bucket. For more information about [Inference engine (Edge Manager agent)](https://docs.aws.amazon.com/sagemaker/latest/dg/edge-device-fleet-about.html).

In [6]:
release_bucket_map = {
    'armv8' : 'sagemaker-edge-release-store-us-west-2-linux-armv8',
    'linux' : 'sagemaker-edge-release-store-us-west-2-linux-x64',
    'win64' : 'sagemaker-edge-release-store-us-west-2-windows-x64',
    'win32' : 'sagemaker-edge-release-store-us-west-2-windows-x86',
}

In [7]:
# In this example, we will run inference on linux platform
release_bucket = release_bucket_map['linux']

To download the artifacts, specify the `VERSION`. The `VERSION` is broken into three components: `<MAJOR_VERSION>.<YYYY-MM-DD>-<SHA-7>`, where:

- MAJOR_VERSION: The release version. The release version is currently set to 1.

- `<YYYY-MM-DD>`: Time stamp of the artifacts release.

- SHA-7: repository commit ID the release is built from.

We suggest you use the latest artifact release time stamp. Use the following to get the latest time stamp.

In [8]:
!aws s3 ls s3://$release_bucket/Releases/ | sort -r

                           PRE 1.20210305.a4bc999/
                           PRE 1.20201218.81f481f/
                           PRE 1.20201207.02d0e97/
2020-12-02 07:31:22          0 


In [9]:
version = "1.20210305.a4bc999"

In [12]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "mkdir /demo",
            "aws s3 cp s3://{}/Releases/{}/{}.tgz demo.tgz".format(release_bucket, version, version),
            "tar -xf demo.tgz -C /demo",
            "cd /demo/bin",
            "chmod +x sagemaker_edge_agent_binary",
            "chmod +x sagemaker_edge_agent_client_example"
        ]
    }
)

In [14]:
ssm_client.get_command_invocation(
    CommandId=response['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '7ff3e274-b16f-4a03-8d28-9730cfc4895b',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-21T07:56:00.803Z',
 'ExecutionElapsedTime': 'PT6.286S',
 'ExecutionEndDateTime': '2021-03-21T07:56:06.803Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Completed 256.0 KiB/9.1 MiB (375.4 KiB/s) with 1 file(s) remaining\rCompleted 512.0 KiB/9.1 MiB (717.0 KiB/s) with 1 file(s) remaining\rCompleted 768.0 KiB/9.1 MiB (1.0 MiB/s) with 1 file(s) remaining  \rCompleted 1.0 MiB/9.1 MiB (1.4 MiB/s) with 1 file(s) remaining    \rCompleted 1.2 MiB/9.1 MiB (1.6 MiB/s) with 1 file(s) remaining    \rCompleted 1.5 MiB/9.1 MiB (1.9 MiB/s) with 1 file(s) remaining    \rCompleted 1.6 MiB/9.1 MiB (2.1 MiB/s) with 1 file(s) remaining    \rCompleted 1.9 MiB/9.1 MiB (2.3 MiB/s) with 1 file(s) remaining    \rCompl

## EC2를 SSH 로 접근
![EC2SSHAccess.png](img/EC2SSHAccess.png)


Get model signing root certificates from release bucket.

In [15]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "cd /demo",
            "mkdir certificates",
            "aws s3 cp s3://{}/Certificates/{}/{}.pem certificates".format(release_bucket, region, region)
        ]
    }
)

In [17]:
ssm_client.get_command_invocation(
    CommandId=response['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '14089667-1da2-4c31-b307-e3a36c534d77',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-21T08:16:18.679Z',
 'ExecutionElapsedTime': 'PT1.747S',
 'ExecutionEndDateTime': '2021-03-21T08:16:19.679Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Completed 1.3 KiB/1.3 KiB (2.9 KiB/s) with 1 file(s) remaining\rdownload: s3://sagemaker-edge-release-store-us-west-2-linux-x64/Certificates/us-east-2/us-east-2.pem to certificates/us-east-2.pem\n',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/14089667-1da2-4c31-b307-e3a36c534d77/i-0aabb20c139dde1af/awsrunShellScript/0.awsrunShellScript/stdout',
 'StandardErrorContent': '',
 'StandardErrorUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-San

Download IoT certificates and private key to EC2 instance. Download models and test images to EC2 instance.

In [20]:
print(root_ca_path)
print(device_cert_path)
print(device_key_path)
print(keras_img_path)
print(keras_model_data)
print(keras_model_package)

s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/iot/AmazonRootCA1.pem
s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/iot/iot.pem.crt
s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/iot/iot_key.pem.key
s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/iot/keras.bmp
s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/compilation-output/keras-model-1.0.tar.gz
keras-model-1.0.tar.gz


In [21]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "cd /demo",
            "mkdir iot-credentials",
            "cd iot-credentials",
            "aws s3 cp " + root_ca_path + " .",  
            "aws s3 cp " + device_cert_path + " .",
            "aws s3 cp " + device_key_path + " .",
            "cd /demo",
            "aws s3 cp " + keras_img_path + " .",
            "aws s3 cp " + keras_model_data + " .",
            "mkdir keras_model",
            "tar -xf " + keras_model_package + " -C keras_model"
        ]
    }
)

In [22]:
ssm_client.get_command_invocation(
    CommandId=response['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': 'd7bd58fb-1a22-45c2-be86-0a17de240442',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-21T08:21:15.578Z',
 'ExecutionElapsedTime': 'PT3.273S',
 'ExecutionEndDateTime': '2021-03-21T08:21:18.578Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Completed 1.2 KiB/1.2 KiB (14.7 KiB/s) with 1 file(s) remaining\rdownload: s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/iot/AmazonRootCA1.pem to ./AmazonRootCA1.pem\nCompleted 1.2 KiB/1.2 KiB (21.5 KiB/s) with 1 file(s) remaining\rdownload: s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/iot/iot.pem.crt to ./iot.pem.crt\nCompleted 1.6 KiB/1.6 KiB (21.3 KiB/s) with 1 file(s) remaining\rdownload: s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/iot/iot_key.pem.key to ./iot_key.pem.key\nComp

#### Configure sagemaker edge manager agent

Generate sagemaker edge agent configuration file. 

In [31]:
print("device_name: ", device_name)
print("device_fleet_name: ", device_fleet_name)
print("region: ", region)
print("endpoint: ", endpoint)
print("bucket: ", bucket)

device_name:  sagemaker-edge-demo-device1616312380
device_fleet_name:  demo2device2fleet1616312351
region:  us-east-2
endpoint:  https://c1oc04uiwvbz1s.credentials.iot.us-east-2.amazonaws.com/role-aliases/SageMakerEdge-demo2device2fleet1616312351/credentials
bucket:  sagemaker-us-east-2-057716757052


In [32]:
sagemaker_edge_config = {
    "sagemaker_edge_core_device_uuid": device_name,
    "sagemaker_edge_core_device_fleet_name": device_fleet_name,
    "sagemaker_edge_core_capture_data_buffer_size": 30,
    "sagemaker_edge_core_capture_data_batch_size": 10,
    "sagemaker_edge_core_capture_data_push_period_seconds": 4,
    "sagemaker_edge_core_folder_prefix": "demo_capture",
    "sagemaker_edge_core_region": region,
    "sagemaker_edge_core_root_certs_path": "/demo/certificates",
    "sagemaker_edge_provider_aws_ca_cert_file": "/demo/iot-credentials/AmazonRootCA1.pem",
    "sagemaker_edge_provider_aws_cert_file": "/demo/iot-credentials/iot.pem.crt",
    "sagemaker_edge_provider_aws_cert_pk_file": "/demo/iot-credentials/iot_key.pem.key",
    "sagemaker_edge_provider_aws_iot_cred_endpoint": endpoint,
    "sagemaker_edge_provider_provider": "Aws",
    "sagemaker_edge_provider_s3_bucket_name": bucket,
    "sagemaker_edge_core_capture_data_destination": "Cloud"
}

In [33]:
edge_config_file = open("sagemaker_edge_config.json", "w")
json.dump(sagemaker_edge_config, edge_config_file, indent = 6)
edge_config_file.close()

In [34]:
! head -n100 sagemaker_edge_config.json

{
      "sagemaker_edge_core_device_uuid": "sagemaker-edge-demo-device1616312380",
      "sagemaker_edge_core_device_fleet_name": "demo2device2fleet1616312351",
      "sagemaker_edge_core_capture_data_buffer_size": 30,
      "sagemaker_edge_core_capture_data_batch_size": 10,
      "sagemaker_edge_core_capture_data_push_period_seconds": 4,
      "sagemaker_edge_core_folder_prefix": "demo_capture",
      "sagemaker_edge_core_region": "us-east-2",
      "sagemaker_edge_core_root_certs_path": "/demo/certificates",
      "sagemaker_edge_provider_aws_ca_cert_file": "/demo/iot-credentials/AmazonRootCA1.pem",
      "sagemaker_edge_provider_aws_cert_file": "/demo/iot-credentials/iot.pem.crt",
      "sagemaker_edge_provider_aws_cert_pk_file": "/demo/iot-credentials/iot_key.pem.key",
      "sagemaker_edge_provider_aws_iot_cred_endpoint": "https://c1oc04uiwvbz1s.credentials.iot.us-east-2.amazonaws.com/role-aliases/SageMakerEdge-demo2device2fleet1616312351/credentials",
      "sagemaker_edge_provid

Upload sagemaker edge agent configure to S3 bucket.

In [35]:
config_path = sess.upload_data('sagemaker_edge_config.json', bucket, iot_folder)
print(config_path)

s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/iot/sagemaker_edge_config.json


Download sagemaker edge agent configure file to EC2 instance.

In [36]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "aws s3 cp " + config_path + ' /demo'
        ]
    }
)

In [37]:
ssm_client.get_command_invocation(
    CommandId=response['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': 'fb296cf4-7ed7-4db4-8c4e-ce396c2c125c',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-21T08:32:33.774Z',
 'ExecutionElapsedTime': 'PT0.734S',
 'ExecutionEndDateTime': '2021-03-21T08:32:33.774Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Completed 1.1 KiB/1.1 KiB (11.6 KiB/s) with 1 file(s) remaining\rdownload: s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/iot/sagemaker_edge_config.json to ../../../../demo/sagemaker_edge_config.json\n',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/fb296cf4-7ed7-4db4-8c4e-ce396c2c125c/i-0aabb20c139dde1af/awsrunShellScript/0.awsrunShellScript/stdout',
 'StandardErrorContent': '',
 'StandardErrorUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east

### /demo/certificates/us-east-2.pem 변경

In [39]:
cert_permission_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    TimeoutSeconds=24*60*60,
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo/certificates",
            "sudo chmod 444 us-east-2.pem",
        ]
    }
)

In [40]:
ssm_client.get_command_invocation(
    CommandId=cert_permission_out['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '338e22c3-9151-4e6b-adcb-b39ac989aae1',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-21T08:37:33.555Z',
 'ExecutionElapsedTime': 'PT0.044S',
 'ExecutionEndDateTime': '2021-03-21T08:37:33.555Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': '',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/338e22c3-9151-4e6b-adcb-b39ac989aae1/i-0aabb20c139dde1af/awsrunShellScript/0.awsrunShellScript/stdout',
 'StandardErrorContent': '',
 'StandardErrorUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/338e22c3-9151-4e6b-adcb-b39ac989aae1/i-0aabb20c139dde1af/awsrunShellScript/0.awsrunShellScript/stderr',
 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
  'CloudWatchO

![Permission2us-ease-2-pem.png](img/Permission2us-ease-2-pem.png)

#### Launch Sagemaker Edge Agent

```
cd /demo
rm -f /tmp/sagemaker_edge_agent_example.sock
./bin/sagemaker_edge_agent_binary -a /tmp/sagemaker_edge_agent_example.sock -c sagemaker_edge_config.json
```
![LaunchEdgeAgent.png](img/LaunchEdgeAgent.png)


## 옵션 (SSM 으로 실행하는 방법)

In [69]:
# agent_out = ssm_client.send_command(
#     InstanceIds=[instance_id],
#     DocumentName="AWS-RunShellScript",
#     TimeoutSeconds=24*60*60,
#     OutputS3BucketName=bucket,
#     OutputS3KeyPrefix=folder,
#     Parameters={
#         'commands':[
#             "cd /demo",
#             "rm -f /tmp/sagemaker_edge_agent_example.sock",
#             "./bin/sagemaker_edge_agent_binary -a /tmp/sagemaker_edge_agent_example.sock -c sagemaker_edge_config.json" 
#         ]
#     }
# )

In [41]:
# ssm_client.get_command_invocation(
#     CommandId=agent_out['Command']['CommandId'],
#     InstanceId=instance_id,
# )

### Load Model

In this section, we show the model management capabilities offered by SageMaker Edge Manager. We will load the two compiled and packaged models with the SageMaker Edge Agent. This keeps both models ready to run inference with. As you will see, once the models are loaded you can run multiple inferences as many times as necessary until the models are unloaded. This relieves the client applications from the logic and operational burden of managing them separately. These models are now simply an API away from running inference with.

When loading the model with SageMaker Edge Agent, the argument to the API points the Agent to a directory containing the packaged model (without any extraneous files within the directory). 

#### Load keras model

`keras_model` is the path containing the packaged model in this notebook. `demo-keras` is the name given to this model. This name will be used later to refer to this model for, making predictions, capturing data, unload.

In [42]:
load_keras_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example LoadModel keras_model demo-keras"
        ]
    }
)

In [43]:
ssm_client.get_command_invocation(
    CommandId=load_keras_model_out['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '82308b43-b4a2-4639-a6f9-21db61d09d32',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-21T08:44:18.820Z',
 'ExecutionElapsedTime': 'PT0.455S',
 'ExecutionEndDateTime': '2021-03-21T08:44:18.820Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Model demo-keras located at /demo/keras_model loaded\nLoadModel succeeded\n',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/82308b43-b4a2-4639-a6f9-21db61d09d32/i-0aabb20c139dde1af/awsrunShellScript/0.awsrunShellScript/stdout',
 'StandardErrorContent': '',
 'StandardErrorUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/82308b43-b4a2-4639-a6f9-21db61d09d32/i-0aabb20c139dde1af/awsrunShellScript/0.awsrunShellScript/stderr

### List Models

This API simply lists all the models and their names that are loaded with SageMaker Edge Agent. Note that the names shown here are same as the ones provided during the LoadModel in the previous sections.

In [44]:
list_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example ListModels"
        ]
    }
)

In [45]:
ssm_client.get_command_invocation(
    CommandId=list_model_out['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '165f515e-c271-44e1-ba78-19d4f3d4c91b',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-21T08:44:44.880Z',
 'ExecutionElapsedTime': 'PT0.101S',
 'ExecutionEndDateTime': '2021-03-21T08:44:44.880Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'There are 1 models\nModel 0  demo-keras\nListModels succeeded\n',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/165f515e-c271-44e1-ba78-19d4f3d4c91b/i-0aabb20c139dde1af/awsrunShellScript/0.awsrunShellScript/stdout',
 'StandardErrorContent': '',
 'StandardErrorUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/165f515e-c271-44e1-ba78-19d4f3d4c91b/i-0aabb20c139dde1af/awsrunShellScript/0.awsrunShellScript/stderr',
 'CloudWa

### Run Predict

In this API, we pass the model name, input data file that will be directly fed into the neural network, input tensor name that was passed earlier during the compilation phase, along with it's size and shape.

#### Run prediction on keras model

In [46]:
keras_predict_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example Predict demo-keras keras.bmp input_1 224 224 3"
        ]
    }
)

In [47]:
ssm_client.get_command_invocation(
    CommandId=keras_predict_out['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '38cfcbfe-5697-4598-9aa8-98738af2c5e7',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-21T08:45:06.804Z',
 'ExecutionElapsedTime': 'PT0.166S',
 'ExecutionEndDateTime': '2021-03-21T08:45:06.804Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Done reading the image\nPredict succeeded\nFlattened RAW Output Tensor:1\n9.15062e-05 0.000100904 5.23973e-05 7.0895e-05 0.000303288 6.65647e-06 5.73232e-05 0.000241128 7.62334e-05 7.42452e-05 0.000110309 4.28148e-05 7.65651e-05 0.000134373 0.000126301 0.000141554 0.000173317 3.4594e-05 6.91597e-05 3.83557e-05 0.000239026 4.7591e-05 7.32565e-05 0.000167224 8.44282e-05 8.91072e-05 0.000395355 0.000366951 0.000141887 1.91341e-05 0.000868919 0.000156764 5.34357e-05 3.37808e-05 6.51875e-05 0.00017874 0.000960836 0.000114277 0.000148988 0.000183138

### Capture Data

Capture the inputs and outputs of an inference call to cloud or disk. The specific parameters were configured earlier in the config file. 

In [48]:
keras_capture_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example PredictAndCapture demo-keras keras.bmp input_1 224 224 3"
        ]
    }
)

In [49]:
output_s3_path = f's3://{bucket}/{folder}'
print(output_s3_path)
# ! aws s3 ls {output_s3_path} --recursive


s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge


In [50]:
ssm_client.get_command_invocation(
    CommandId=keras_capture_out['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '6229fc95-2b26-4905-8d9b-07af9976e19b',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-21T08:45:39.828Z',
 'ExecutionElapsedTime': 'PT0.272S',
 'ExecutionEndDateTime': '2021-03-21T08:45:39.828Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Done reading the image\nPredict succeeded\nCapture Data succeeded\nFlattened RAW Output Tensor:1\n9.15062e-05 0.000100904 5.23973e-05 7.0895e-05 0.000303288 6.65647e-06 5.73232e-05 0.000241128 7.62334e-05 7.42452e-05 0.000110309 4.28148e-05 7.65651e-05 0.000134373 0.000126301 0.000141554 0.000173317 3.4594e-05 6.91597e-05 3.83557e-05 0.000239026 4.7591e-05 7.32565e-05 0.000167224 8.44282e-05 8.91072e-05 0.000395355 0.000366951 0.000141887 1.91341e-05 0.000868919 0.000156764 5.34357e-05 3.37808e-05 6.51875e-05 0.00017874 0.000960836 0.000114277

In [51]:
s3_out='s3://sagemaker-us-east-2-057716757052/DEMO-Sagemaker-Edge/87ed1414-5da6-40d5-acb5-b6f6ec6191c9/i-0de031526f9170fb6/awsrunShellScript/0.awsrunShellScript/stdout'
!aws s3 cp {s3_out} .

download: s3://sagemaker-us-east-2-057716757052/DEMO-Sagemaker-Edge/87ed1414-5da6-40d5-acb5-b6f6ec6191c9/i-0de031526f9170fb6/awsrunShellScript/0.awsrunShellScript/stdout to ./stdout


### Unload Model

After unloading a model, the same name can be reused for future `LoadModel` APIs calls.

In [52]:
unload_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example UnloadModel demo-keras"
        ]
    }
)

In [53]:
ssm_client.get_command_invocation(
    CommandId=unload_model_out['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '07354c70-e32a-4d00-8153-b67acad4e886',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-21T08:47:09.006Z',
 'ExecutionElapsedTime': 'PT0.12S',
 'ExecutionEndDateTime': '2021-03-21T08:47:09.006Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Model demo-keras has been unloaded\nUnLoadModel succeeded\n',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/07354c70-e32a-4d00-8153-b67acad4e886/i-0aabb20c139dde1af/awsrunShellScript/0.awsrunShellScript/stdout',
 'StandardErrorContent': '',
 'StandardErrorUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/07354c70-e32a-4d00-8153-b67acad4e886/i-0aabb20c139dde1af/awsrunShellScript/0.awsrunShellScript/stderr',
 'CloudWatchOu

## Clean Up

Stop the Agent

ssm_client.cancel_command(
    CommandId=agent_out['Command']['CommandId'],
    InstanceIds=[instance_id]
)

Stop the EC2 instance

ec2_client.stop_instances(
    InstanceIds=[instance_id]
)

Detach and delete policy

iot_client.detach_policy(
    policyName=policy_name,
    target=iot_cert['certificateArn']
)

iot_client.delete_policy(
    policyName=policy_name
)

Deregister device and delete device fleet

sagemaker_client.deregister_devices(
    DeviceFleetName=device_fleet_name,
    DeviceNames=[device_name]
)

sagemaker_client.delete_device_fleet(
    DeviceFleetName=device_fleet_name
)

## Appendix

### (Optional)Install CloudWatch Agent 

In [54]:
CW_log_config = {
      "agent": {
        "metrics_collection_interval": 10,
        "logfile": "/opt/aws/amazon-cloudwatch-agent/logs/amazon-cloudwatch-agent.log"
      },
      "logs": {
        "logs_collected": {
          "files": {
            "collect_list": [
              {
                "file_path": "/opt/aws/amazon-cloudwatch-agent/logs/amazon-cloudwatch-agent.log",
                "log_group_name": "amazon-cloudwatch-agent.log",
                "log_stream_name": "amazon-cloudwatch-agent.log",
                "timezone": "UTC"
              },
              {
                "file_path": "/opt/aws/amazon-cloudwatch-agent/logs/test.log",
                "log_group_name": "test.log",
                "log_stream_name": "test.log",
                "timezone": "Local"
              }
            ]
          }
        },
        "log_stream_name": "my_log_stream_name",
        "force_flush_interval" : 15
      }
}

In [55]:
CW_file = open("cloudwatch.json", "w") 
json.dump(CW_log_config, CW_file, indent = 6) 
CW_file.close() 

In [56]:
CW_config_path = sess.upload_data('cloudwatch.json', bucket, iot_folder)
print("CW_config_path: ", CW_config_path)

CW_config_path:  s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/iot/cloudwatch.json


In [58]:
cw_download_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "aws s3 cp " + CW_config_path + " /opt/aws/amazon-cloudwatch-agent/etc/amazon-cloudwatch-agent.json"
        ]
    }
)

In [59]:
ssm_client.get_command_invocation(
    CommandId=cw_download_out['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '8478abf5-781d-4f8f-a3d2-e599f7a58e6f',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-21T08:50:31.722Z',
 'ExecutionElapsedTime': 'PT0.682S',
 'ExecutionEndDateTime': '2021-03-21T08:50:31.722Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Completed 1.2 KiB/1.2 KiB (24.6 KiB/s) with 1 file(s) remaining\rdownload: s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/iot/cloudwatch.json to ../../../../opt/aws/amazon-cloudwatch-agent/etc/amazon-cloudwatch-agent.json\n',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/8478abf5-781d-4f8f-a3d2-e599f7a58e6f/i-0aabb20c139dde1af/awsrunShellScript/0.awsrunShellScript/stdout',
 'StandardErrorContent': '',
 'StandardErrorUrl': 'https://s3.us-east-2.amazonaws

In [61]:
cw_dpkg_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "wget https://s3.amazonaws.com/amazoncloudwatch-agent/ubuntu/amd64/latest/amazon-cloudwatch-agent.deb",
            "sudo dpkg -i -E ./amazon-cloudwatch-agent.deb",
        ]
    }
)

Install Cloud Watch Agent to SSM agent.

In [64]:
ssm_client.get_command_invocation(
    CommandId=cw_dpkg_out['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '9cbabb4f-7117-403b-ba0c-5367cdc0c63d',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-21T08:55:15.506Z',
 'ExecutionElapsedTime': 'PT7.292S',
 'ExecutionEndDateTime': '2021-03-21T08:55:22.506Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Selecting previously unselected package amazon-cloudwatch-agent.\n(Reading database ... 167561 files and directories currently installed.)\nPreparing to unpack ./amazon-cloudwatch-agent.deb ...\ncreate group cwagent, result: 0\ncreate user cwagent, result: 0\nUnpacking amazon-cloudwatch-agent (1.247347.5b250583-1) ...\nSetting up amazon-cloudwatch-agent (1.247347.5b250583-1) ...\nProcessing triggers for ureadahead (0.100.0-21) ...\n',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagema

In [65]:
install_cw_out = ssm_client.send_command(
    DocumentName="AWS-ConfigureAWSPackage",
    DocumentVersion='1',
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Targets=[
        {
            'Key': 'InstanceIds',
            'Values': [instance_id]
        },
    ],
    TimeoutSeconds=600,
    Parameters={
        'action': ['Install'],
        "name": ["AmazonCloudWatchAgent"]
    },
    MaxConcurrency='50',
    MaxErrors='0'
)

In [67]:
ssm_client.get_command_invocation(
    CommandId=install_cw_out['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': 'd9b87b78-5a09-4a03-8601-51e53b96bcc6',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-ConfigureAWSPackage',
 'DocumentVersion': '1',
 'PluginName': 'configurePackage',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-21T08:56:39.232Z',
 'ExecutionElapsedTime': 'PT3.48S',
 'ExecutionEndDateTime': '2021-03-21T08:56:42.232Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Initiating arn:aws:ssm:::package/AmazonCloudWatchAgent 1.247347.5b250583 install\nPlugin aws:runShellScript ResultStatus Success\ninstall output: Running sh install.sh\n\nSuccessfully installed arn:aws:ssm:::package/AmazonCloudWatchAgent 1.247347.5b250583\n',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/d9b87b78-5a09-4a03-8601-51e53b96bcc6/i-0aabb20c139dde1af/awsconfigurePackage/configurePackage/stdout',
 'StandardErrorContent': 'install errors: dpkg: version 1.247347

In [68]:
cloudwatch_response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    CloudWatchOutputConfig={
        'CloudWatchOutputEnabled': True
    },
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/neo_agent_binary -a /tmp/sagemaker_edge_agent_example.sock -c neo_config.json" 
        ]
    }
)

To debug with CloudWatch, add a paramater `CloudWatchOutputConfig` to `send_command`
```
CloudWatchOutputConfig={
    'CloudWatchOutputEnabled': True
}
```

Example:
```
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    CloudWatchOutputConfig={
        'CloudWatchOutputEnabled': True
    },
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/neo_agent_binary -a /tmp/sagemaker_edge_agent_example.sock -c neo_config.json" 
        ]
    }
)
```

Running log can be found in cloud watch log group `/aws/ssm/AWS-RunShellScript`

In [70]:
ssm_client.get_command_invocation(
    CommandId=cloudwatch_response['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '415570ea-194a-450f-bb0f-102e686015a3',
 'InstanceId': 'i-0aabb20c139dde1af',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 127,
 'ExecutionStartDateTime': '2021-03-21T08:58:10.907Z',
 'ExecutionElapsedTime': 'PT3.155S',
 'ExecutionEndDateTime': '2021-03-21T08:58:13.907Z',
 'Status': 'Failed',
 'StatusDetails': 'Failed',
 'StandardOutputContent': '',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/415570ea-194a-450f-bb0f-102e686015a3/i-0aabb20c139dde1af/awsrunShellScript/0.awsrunShellScript/stdout',
 'StandardErrorContent': '/var/lib/amazon/ssm/i-0aabb20c139dde1af/document/orchestration/415570ea-194a-450f-bb0f-102e686015a3/awsrunShellScript/0.awsrunShellScript/_script.sh: 2: /var/lib/amazon/ssm/i-0aabb20c139dde1af/document/orchestration/415570ea-194a-450f-bb0f-102e686015a3/awsrunShellScript/0.awsrunShellScript/_scrip

![cw_error.png](img/cw_error.png)